In [77]:
import cv2
import numpy as np
import os
import glob
import dlib
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from face_encodings import Base, Face

In [78]:
# database connection
engine = create_engine('sqlite:///face_encodings.db')
Session = sessionmaker(bind=engine)
session = Session()

In [79]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(r'C:\Users\adam.bloebaum\.vscode\driveline\cv\mmod_human_face_detector.dat')
face_rec_model = dlib.face_recognition_model_v1(r'C:\Users\adam.bloebaum\.vscode\driveline\cv\dlib_face_recognition_resnet_model_v1.dat')
shape_predictor = dlib.shape_predictor(r"C:\Users\adam.bloebaum\.vscode\driveline\cv\shape_predictor_68_face_landmarks.dat")

video_directory = r"C:\Users\adam.bloebaum\Desktop\face_scans"
video_path = r"C:\Users\adam.bloebaum\Desktop\face_scans\IMG_1970.MOV"

In [80]:
# extract every Nth frame from the video
def extract_frames(video_path, frame_skip=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frames.append(frame)
        frame_count += 1

    cap.release()
    return frames

In [81]:
# detect faces
def detect_faces(frames, cnn_face_detector):
    detected_faces = []

    for frame in frames:
        # convert the OpenCV BGR image to RGB
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        faces = cnn_face_detector(rgb_image, 1)

        for face in faces:
            x, y, w, h = face.rect.left(), face.rect.top(), face.rect.width(), face.rect.height()
            detected_faces.append(rgb_image[y:y+h, x:x+w])

    return detected_faces

In [82]:
# apply distortions to faces for robustness
def distort_faces(faces, rotate=True, add_noise=True, flip=True):
    distorted_faces = []

    for face in faces:
        if rotate:
            # rotate the face by a random angle between -30 and 30 degrees
            (h, w) = face.shape[:2]
            center = (w // 2, h // 2)
            angle = np.random.uniform(-30, 30)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            face = cv2.warpAffine(face, M, (w, h))

        if add_noise:
            # add random noise to the face
            noise = np.random.normal(0, 15, face.shape).astype(np.uint8)
            face = cv2.add(face, noise)

        if flip:
            # horizontally flip the face
            face = cv2.flip(face, 1)

        distorted_faces.append(face)

    return distorted_faces

In [83]:
def extract_face_encodings(face):
    # convert the OpenCV BGR image to RGB
    rgb_image = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    # initialize the Dlib face detector and shape predictor
    face_detector = dlib.get_frontal_face_detector()

    # detect faces
    detected_faces = face_detector(rgb_image, 1)
    if len(detected_faces) == 0:
        # no faces found in the image, return None or an appropriate value
        return None

    # get the landmarks/parts for the face
    shape = shape_predictor(rgb_image, detected_faces[0])

    # align and crop the face using the landmarks
    face_chip = dlib.get_face_chip(rgb_image, shape)

    # compute the face descriptor
    face_descriptor = face_rec_model.compute_face_descriptor(face_chip)

    # convert the descriptor to a numpy array, then to a list, and serialize to JSON
    encoding = np.array(face_descriptor).tolist()
    return json.dumps(encoding)

In [84]:
def store_face_encoding(session, encoding):
    new_face = Face(encoding=encoding)
    session.add(new_face)
    session.commit()

In [85]:
# process each video
def process_video(video_path):
    frames = extract_frames(video_path)
    print(len(frames), "frames extracted.")
    faces = detect_faces(frames, cnn_face_detector)
    print(len(faces), "faces detected.")
    rotated_faces = distort_faces(faces, rotate=True, add_noise=False, flip=False)
    noisy_faces = distort_faces(faces, rotate=False, add_noise=True, flip=False)
    flipped_faces = distort_faces(faces, rotate=False, add_noise=False, flip=True)
    combined_faces = faces + rotated_faces + noisy_faces + flipped_faces
    print(len(combined_faces), "total faces.")
    
    processed_faces = 0
    for face in combined_faces:
        encoding = extract_face_encodings(face)
        if encoding is not None:
            store_face_encoding(session, encoding)
            processed_faces += 1
    print("Successfully processed", processed_faces, "face encodings to the database.")

In [86]:
for video_file in glob.glob(os.path.join(video_directory, '*.mov')):
    process_video(video_file)

5 frames extracted.
5 faces detected.
5 faces rotated.
5 faces noised.
5 faces flipped.
20 total faces.
Successfully processed 10 face encodings to the database.


In [90]:
# query the database for binary data
faces = session.query(Face.image_data).all()

# display the images
for face in faces:
    # convert binary data to NumPy array and then to image
    nparr = np.frombuffer(face.image_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # display the image using OpenCV
    cv2.imshow('Face', img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [76]:
# delete all records in the 'Face' table
session.query(Face).delete()

# commit the changes
session.commit()